In [ ]:
import pandas as pd
import numpy as np

## Import all of the data

In [33]:
#Import WDI data
data=pd.read_csv("/Users/leoss/Downloads/P_Data_Extract_From_WDI_Database_Archives-1/e0ed2ed0-d862-46c9-9602-14890114e90f_Series - Metadata.csv")
data.drop(columns=['Series Code', 'Version Name', 'Version Code'], inplace=True)
pop=pd.read_csv("/Users/leoss/Downloads/P_Data_Extract_From_World_Development_Indicators/831d0f95-9643-4707-81f4-383d280f9ba9_Data.csv")

#Import Trade data on complexity
trade=pd.read_csv("/Users/leoss/Downloads/growth_proj_eci_rankings.csv")
columns1 = ["country_id",'growth_proj', 'in_rankings', 'eci_sitc', 'eci_rank_sitc','eci_rank_hs92', 'eci_hs12', 'eci_rank_hs12']
trade.drop(columns1, inplace=True, axis=1) 

#Import imf dataset
ins = ["COUNTRY.ID", "COUNTRY", "INDICATOR", "TIME_PERIOD", 
       "OBS_VALUE", "SCALE.ID", "PRIMARY_DOMESTIC_CURRENCY"]
imf = pd.read_csv(
    "/Users/leoss/Downloads/WorldEconomicOutlook-1.csv", 
    usecols=ins
)
imf = imf[ins]

## Standardise WDI

In [34]:
#Standardise them into long format
id_cols = [
    'Country Name', 'Country Code', 'Series Name'
]
Values=[f"{i} "+f"[YR{i}]" for i in range(1991,2025)]
data_gen = data.melt(
    id_vars=id_cols, 
    value_vars=Values,
    var_name='Year',  
    value_name='Value'    
)
pop_long=pop.melt(
    id_vars=id_cols, 
    value_vars=Values,
    var_name='Year',  
    value_name='Value'    
)
data_long = pd.concat([data_gen, pop_long], ignore_index=True)

#Coerce all non numerics into Missing data 
data_long['Value'] = pd.to_numeric(data_long['Value'], errors='coerce')
data_long['Year'] = data_long['Year'].str.extract(r'(\d{4})').astype(int)
final_cols = id_cols + ['Year', 'Value']
data_long["Country Code"].dropna(inplace=True)
df_long = data_long[final_cols]

## Standardise AGL and IMF

In [35]:
#Standardise names and order
trade["Series Name"]="Economic Complexity"
trade = trade.rename(columns={
    "country_iso3_code": "Country Code",
    "Country":"Country Name",
    "year": "Year",
    "eci_hs92": "Value"
})
new_order = ['Country Code','Series Name', 'Year', 'Value' ]
trade = trade[new_order]

#Dropping all observations without an associated time period and rename 
imf["TIME_PERIOD"].dropna(inplace=True)
imf = imf.rename(columns={
    "COUNTRY.ID": "Country Code",
    "COUNTRY":"Country Name",
    "TIME_PERIOD": "Year",
    "INDICATOR":"Series Name",
    "OBS_VALUE": "Value"
})

#Extract value scaled
imf["Value"]=imf["Value"]*10**(imf["SCALE.ID"])

#Drop those as not needed anymore
imf.drop(columns=["SCALE.ID","PRIMARY_DOMESTIC_CURRENCY"], inplace=True)

#Choose variables of interest
raw_list = [
    "Gross domestic product (GDP), Per capita, purchasing power parity (PPP) international dollar, ICP benchmarks 2017-2021",
    "Revenue, General government, Percent of GDP"
]
variables_to_keep = [s.strip() for s in raw_list]
imf = imf[imf['Series Name'].isin(variables_to_keep)].query("Year>1990 & Year<2025")

## Standardise unique country identifiers across all the data.

In [36]:
#Get unique country codes from each database
codes_cleaned = set(df_long['Country Code'].unique())
codes_trade = set(trade['Country Code'].unique())
codes_imf = set(imf['Country Code'].unique())
common_codes = codes_cleaned.intersection(codes_trade, codes_imf)
print(f"Found {len(common_codes)} countries common to all datasets.")

#Filter the original dataframes to keep only the common codes
df_cleaned_filtered = df_long[df_long['Country Code'].isin(common_codes)].copy()
trade_filtered = trade[trade['Country Code'].isin(common_codes)].copy()
imf_filtered = imf[imf['Country Code'].isin(common_codes)].copy()

#Final merge
final_dataset = pd.concat([df_cleaned_filtered, trade_filtered, imf_filtered], ignore_index=True)
final_dataset.sort_values(by=['Country Code', 'Year'], inplace=True)
final_dataset.dropna(subset=['Country Code'], inplace=True)

Found 144 countries common to all datasets.


In [37]:
#Very important, here we put all the variables that need to be dropped before the final checks
series_to_drop = [
'Tax revenue (% of GDP)', 
'Central government debt, total (% of GDP)', 
"Current account balance (credit less debit), US dollar",
"GDP per capita (current US$)"]
final_dataset = final_dataset[~final_dataset['Series Name'].isin(series_to_drop)].query("Year>1990 & Year<2025")

clmns_cluster=['Economic Complexity', 
       'Gross domestic product (GDP), Per capita, purchasing power parity (PPP) international dollar, ICP benchmarks 2017-2021',
       'Mineral rents (% of GDP)', 'Natural gas rents (% of GDP)',
       'Oil rents (% of GDP)', 
       'Revenue, General government, Percent of GDP','Rents']

## Fully standardise unique country identifiers

In [38]:
"""##The final dataset will struggle with Name-Code matchings
country_name_map = final_dataset.groupby('Country Code')['Country Name'].unique().reset_index()
def clean_name_list(name_list):
    clean_list = []
    for name in name_list:
        if pd.notna(name):
            clean_list.append(str(name))
    return clean_list

country_name_map['Cleaned_Names'] = country_name_map['Country Name'].apply(clean_name_list)
country_name_map['Valid_Name_Count'] = country_name_map['Cleaned_Names'].apply(len)

# Check for duplicate names for the same Country Code
conflicts_found = False
no_name_count = 0
single_name_count = 0

for index, row in country_name_map.iterrows():
    code = row['Country Code']
    names = row['Cleaned_Names']
    count = row['Valid_Name_Count']
    if count > 1:
        print(f"Code:'{code}' maps to multiple names: {names}")
        conflicts_found = True
    elif count == 0:
        print(f"Code': {code}' only has missing (NaN) Country Names associated with it.")
        no_name_count += 1
    else:
        single_name_count += 1

print(f"\nTotal codes with one unique name: {single_name_count}")
print(f"Total codes with no valid name: {no_name_count}")
print(f"Total unique codes checked: {len(country_name_map)}")
"""
#Manually fixing the names:
mapping={"AFG":'Afghanistan',
         'ARM':'Armenia',
         'AZE':'Azerbaijan',
         'BHR':'Bahrain',
         'BLR':'Belarus',
         'CHN':'China',
         'CIV':"Cote d'Ivoire",
         'COD':'Congo Dem. Rep.',
         'COG':'Congo',
         'CZE':"Czechia",
         'EGY':"Egypt",
         'EST':'Estonia',
         'ETH':'Ethiopia',
         'GNQ':'Equatorial Guinea',
         'HKG':'Hong Kong',
         'HRV':'Croatia',
         'IRN':'Iran',
         'KAZ':'Kazakhstan',
         'LAO':'Laos',
         "LTU":'Lithuania',
         'LVA':'Latvia',
         'MDA':'Moldova',
         'KOR':'South Korea',
         'KAZ':'Kazakhstan',
         'MDG':'Madagascar',
         'MKD':'North Macedonia',
         'MOZ':'Mozambique',
         'MRT':'Mauritania',
         'NLD':'Netherlands',
         'POL':'Poland',
         'RUS':'Russia',
         'SRB':'Serbia',
         'SVN':'Slovenia',
         'SWZ':'Eswatini',
         'TJK':'Tajikistan',
         'TUR':'Turkey',
         'TWN':'Taiwan',
         'TZA':"Tanzania",
         'UZB':'Uzbekistan',
         'VEN':'Venezuela',
         "VNM":"Vietnam",
         'YEM':'Yemen'
         }

# Generate name-code map
valid_names = final_dataset.dropna(subset=['Country Name'])
code_to_name_map_df = valid_names.drop_duplicates(subset=['Country Code'])
clean_map_dict = pd.Series(
    code_to_name_map_df['Country Name'].values, 
    index=code_to_name_map_df['Country Code']
).to_dict()

clean_map_dict.update(mapping)
final_dataset['Country Name'] = final_dataset['Country Code'].map(clean_map_dict)


## Turn it into a comprehgensive dataset 

In [39]:
#Identify all unique items
unique_countries = final_dataset['Country Code'].unique()
unique_series = final_dataset['Series Name'].unique()
unique_years =  final_dataset['Year'].unique()

# Create a "Skeleton" DataFrame, with every possible combination of Country, Series, and Year
idx = pd.MultiIndex.from_product(
    [unique_countries, unique_series, unique_years], 
    names=['Country Code', 'Series Name', 'Year']
)
skeleton_df = pd.DataFrame(index=idx).reset_index()
skeleton_df['Year'] = pd.to_numeric(skeleton_df['Year'], errors='coerce')
final_dataset['Year'] = pd.to_numeric(final_dataset['Year'], errors='coerce')

# Convert both Year columns to numeric to ensure they match format
skeleton_df['Year'] = pd.to_numeric(skeleton_df['Year'], errors='coerce')
final_dataset['Year'] = pd.to_numeric(final_dataset['Year'], errors='coerce')

# Merge your data through left-join
comprehensive_df = pd.merge(
    skeleton_df, 
    final_dataset, 
    on=['Country Code', 'Series Name', 'Year'], 
    how='left'
)
#Fix the "Country Code" column
comprehensive_df['Country Name'] = comprehensive_df['Country Code'].map(clean_map_dict)
comprehensive_df = comprehensive_df.sort_values(by=['Country Name', 'Series Name', 'Year'])
print(f"Original rows: {len(final_dataset)}")
print(f"Comprehensive rows: {len(comprehensive_df)}")

#Calculate the average for every Series/Country pair across the whole dataset
group_means = final_dataset.groupby(['Series Name', 'Country Name'])['Value'].transform('mean')
target_mask = (final_dataset['Value'].isna()) & \
              (group_means < 0.02) & \
              (group_means.notna())
num_filled = target_mask.sum()
final_dataset.loc[target_mask, 'Value'] = 0
#print(f"Done! Successfully filled {num_filled} missing values with 0.")
#print("These were only applied to countries where the variable average was < 0.02.")


Original rows: 47673
Comprehensive rows: 48960


In [40]:
#Filter for your specific period first
df_clean = comprehensive_df.copy()

#Calculate percentage of missing values for each country and drop
country_missing_pct = df_clean.groupby('Country Name')['Value'].apply(lambda x: x.isna().mean())
THRESHOLD = 0.40 
countries_to_drop = country_missing_pct[country_missing_pct > THRESHOLD].index
countries_to_keep = country_missing_pct[country_missing_pct <= THRESHOLD].index
df_clean = df_clean[df_clean['Country Name'].isin(countries_to_keep)]
#print(f"Dropped {len(countries_to_drop)} countries.")
#print(f"Remaining countries: {df_clean['Country Name'].nunique()}")

# If a country has data for 2016 and 2018, this calculates 2017 linearly.
df_clean['Value'] = df_clean.groupby(['Country Name', 'Series Name'])['Value'].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both')
)

print(f"Remaining Missing Values: {df_clean['Value'].isna().sum()}")
print(df_clean.head())

Remaining Missing Values: 238
    Country Code          Series Name    Year Country Name  Value
238          AFG  Economic Complexity  1991.0  Afghanistan -0.451
239          AFG  Economic Complexity  1992.0  Afghanistan -0.451
240          AFG  Economic Complexity  1993.0  Afghanistan -0.451
241          AFG  Economic Complexity  1994.0  Afghanistan -0.451
242          AFG  Economic Complexity  1995.0  Afghanistan -0.451


In [41]:
#Export to CSV
df_clean.to_csv('comprehensive_dataset_2.csv', index=False, encoding='utf-8-sig')
print("Successfully saved to 'comprehensive_dataset_2.csv'")

Successfully saved to 'comprehensive_dataset_2.csv'


## Visualise data

In [42]:
import seaborn as sns
import matplotlib.pyplot as plt

In [43]:
for target_series in final_dataset["Series Name"].unique():

    df_variable= final_dataset[
        (final_dataset["Series Name"] == target_series) & 
        (final_dataset["Year"] > 2014)
    ].copy()
    top_countries = df_variable.groupby('Country Name')['Value'].mean().sort_values(ascending=False).head(10)
    print(f"Top 10 Countries by Average {target_series} (Post-2014):")
    print(top_countries)

""" # Visualize with a Bar Chart
    plt.figure(figsize=(12, 6))
    sns.barplot(x=top_countries.values, y=top_countries.index, palette='viridis')
    plt.title(f"Top 10 Countries: {target_series}\n(Average Post-2014)")
    plt.xlabel("Average Value (% of GDP)")
    plt.ylabel("Country")
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()"""

Top 10 Countries by Average Mineral rents (% of GDP) (Post-2014):
Country Name
Mongolia            9.691451
Congo Dem. Rep.     8.418404
Zambia              8.074911
Mali                6.833144
Kyrgyz Republic     6.670242
Uzbekistan          6.234966
Papua New Guinea    5.563554
Chile               5.385366
Burkina Faso        5.310223
Peru                5.033037
Name: Value, dtype: float64


Top 10 Countries by Average Natural gas rents (% of GDP) (Post-2014):
Country Name
Turkmenistan           8.643183
Qatar                  7.632616
Papua New Guinea       6.418297
Iran                   5.260040
Uzbekistan             5.202735
Trinidad and Tobago    5.100249
Equatorial Guinea      4.524533
Algeria                4.307927
Bahrain                3.376391
Azerbaijan             3.232013
Name: Value, dtype: float64
Top 10 Countries by Average Oil rents (% of GDP) (Post-2014):
Country Name
Iraq                 37.851736
Kuwait               35.736117
Congo                25.855679
Libya                24.939175
Angola               23.179967
Saudi Arabia         22.808302
Oman                 19.757023
Azerbaijan           16.962412
Iran                 16.722606
Equatorial Guinea    15.036217
Name: Value, dtype: float64
Top 10 Countries by Average Total natural resources rents (% of GDP) (Post-2014):
Country Name
Iraq                 38.220536
Kuwait               36.785248

' # Visualize with a Bar Chart\n    plt.figure(figsize=(12, 6))\n    sns.barplot(x=top_countries.values, y=top_countries.index, palette=\'viridis\')\n    plt.title(f"Top 10 Countries: {target_series}\n(Average Post-2014)")\n    plt.xlabel("Average Value (% of GDP)")\n    plt.ylabel("Country")\n    plt.grid(axis=\'x\', alpha=0.3)\n    plt.tight_layout()\n    plt.show()'

In [44]:
for target_series in final_dataset["Series Name"].unique():

    df_variable= final_dataset[final_dataset["Series Name"] == target_series]
    top_countries_list = df_variable.groupby('Country Name')['Value'].mean().sort_values(ascending=False).head(10).index.tolist()
    plot_data = df_variable[df_variable['Country Name'].isin(top_countries_list)].sort_values("Year")

    """plt.figure(figsize=(12, 6))
    sns.lineplot(data=plot_data, x='Year', y='Value', hue='Country Name', marker='o')

    plt.title(f"Time Series: Top 10 Countries for {target_series}")
    plt.ylabel(f"{target_series}")
    plt.xlabel("Year")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Country')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()"""